In [132]:
import pandas as pd
import yfinance as yf
from datetime import datetime as dt
pd.set_option('future.no_silent_downcasting', True)

# Get WACC

In [476]:
def get_risk_free_rate():
    """
    Get the current 10-year Treasury yield (risk-free rate) from an API.
    """
    try:
        tnx = yf.Ticker("^TNX")
        current_yield = tnx.history(period="1d")['Close'].iloc[-1] * 0.01
        return round(current_yield, 3)
    except:
        return 0.04
        
def get_wacc(ticker):
    stock= yf.Ticker(ticker)
    #Get Market Cap
    market_cap = stock.info.get('marketCap', 0)

    #Get Total Debt
    total_debt = stock.balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in stock.balance_sheet else 0

    # Get Beta
    beta = stock.info.get('beta', 1)

    # Get Interest Expense & Total Revenue to estimate Cost of Debt (Rd)
    interest_expense = stock.financials.loc['Interest Expense'].iloc[0] if 'Interest Expense' in stock.financials else 0
    cost_of_debt = abs(interest_expense / total_debt) if total_debt else 0.05  # Default 5% if no data

    # Get Tax Rate
    income_tax = stock.financials.loc['Income Tax Expense'].iloc[0] if 'Income Tax Expense' in stock.financials else 0
    ebit = stock.financials.loc['EBIT'][0] if 'EBIT' in stock.financials else 0
    tax_rate = income_tax / ebit if ebit else 0.21  # Default to 21%

    # Get Risk-Free Rate and Market Return (estimated ~8%)
    risk_free_rate = get_risk_free_rate()
    market_return = 0.08

    #Calculate Cost of Equity (Re) using CAPM
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
    
    # Total Value (V = E + D)
    total_value = market_cap + total_debt

    # Calculate WACC
    wacc = (market_cap / total_value * cost_of_equity) + (total_debt / total_value * cost_of_debt * (1 - tax_rate))

    return round(wacc, 4) 
    
#past 4 year revenue growth
def get_revenue_growth(ticker):
    stock= yf.Ticker(ticker)
    revenue= stock.income_stmt.loc['Total Revenue'].dropna() /1e9
    revenue = revenue.sort_index()

    growth_rates = revenue.pct_change()
    avg_growth = pd.Series(growth_rates[1:]).mean()

    return revenue, avg_growth
    
#5 year revenue projection
def get_revenue_projection(ticker, growth_rate):
    stock= yf.Ticker(ticker)
    revenue_projection = []
    projection_years=[]
    last_year = stock.income_stmt.columns[0].year
    last_year_revenue= stock.income_stmt.loc['Total Revenue'].iloc[0]/1e9
    
    for x in range(5):
        next_year_revenue = (last_year_revenue * (1 + growth_rate))
        revenue_projection.append(round(next_year_revenue, 2))
        last_year_revenue = next_year_revenue
    
        projection_years.append(last_year + (x + 1))
        
    return pd.Series(revenue_projection, projection_years) 

def get_ebit_margin(ticker):
    stock= yf.Ticker(ticker)
    if 'Operating Income' in stock.income_stmt.index:
        operating_income = stock.income_stmt.loc['Operating Income'].dropna()
        revenue = stock.income_stmt.loc['Total Revenue'].dropna()
        operating_margin = operating_income/ revenue
        
        return operating_income/1e9, operating_margin,  operating_margin.mean()
        
    else:
        print("Operating Income not found")
        
def get_ebit_projection(revenue_projection, ebit_margin):
    ebit_projection = []
    
    for revenue in revenue_projection:
        ebit = revenue * ebit_margin
        ebit_projection.append(round(ebit, 2))
    return pd.Series(ebit_projection, revenue_projection.index) 

def get_NOPAT(ebit_projection):
    ebit_after_tax = []
    tax_rate= 0.21

    for ebit in ebit_projection:
        nopat = ebit * (1 - tax_rate)
        ebit_after_tax.append(nopat)
    return pd.Series(ebit_after_tax, ebit_projection.index) 
    
def get_depreciation_and_amortization(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Depreciation And Amortization' in stock.cashflow.index:
        da = stock.cashflow.loc['Depreciation And Amortization'].dropna()/1e9
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        da_rate = (da/revenue).mean()
        return da, da_rate * projection
    else:
        print("Depreciation & Amortization not found")

def get_capex(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Capital Expenditure' in stock.cashflow.index:
        capex = abs(stock.cashflow.loc['Capital Expenditure'].dropna()/1e9)
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        capex_rate = (capex/revenue).mean()
        return capex_rate * projection
    else:
        print("Capital Expenditure not found")

def get_workingcapital(ticker, projection):
    stock= yf.Ticker(ticker)
    if 'Change In Working Capital' in stock.cashflow.index:
        nwc = abs(stock.cashflow.loc['Change In Working Capital'].dropna()/1e9)
        revenue= stock.income_stmt.loc['Total Revenue'].dropna()/1e9
        nwc_rate = (nwc/revenue).mean()
        return nwc_rate * projection
    else:
        print("Change In Working Capital not found")
        
def get_fcf(NOPAT, da, capex, nwc):
    fcf = NOPAT + da - capex - nwc
    
    return fcf

def get_fcf_pv(fcf_projection, wacc):
    discounted = []
    
    for t, fcf in enumerate(fcf_projection, start=1):
        pv = fcf/((1 + wacc) ** t)
        discounted.append(pv)
        
    return pd.Series(discounted, fcf_projection.index)  

def get_terminal_value(fcf_projection, wacc):
    terminal_growth = 0.02
    final_year_fcf = fcf_projection.iloc[-1]
    
    terminal_value = (final_year_fcf * (1 + terminal_growth))/(wacc - terminal_growth)
    
    return terminal_value

def get_pv_tv(tv, wacc):
    n = 5
    pv_tv= tv/((1 + wacc) ** n)

    return pv_tv

def get_enterprise_value(pv_fcf, pv_tv):
    pv_fcf_sum = pv_fcf.sum()
    
    ev = pv_fcf_sum + pv_tv

    return ev
def get_net_debt(ticker):  
    stock= yf.Ticker(ticker)
    cash = stock.balance_sheet.loc['Cash And Cash Equivalents'].iloc[0] 
    debt = stock.balance_sheet.loc['Long Term Debt'].iloc[0] 
    net_debt = debt - cash 

    return cash/1e9, debt/1e9, net_debt/1e9

def get_equity_value(ev, net_debt):
    return ev - net_debt

def get_implied_share_price(ticker, equity_value):
    stock= yf.Ticker(ticker)
    share_outstanding = stock.info['sharesOutstanding']/1e9
    per_share_value = equity_value / share_outstanding

    return share_outstanding, per_share_value

def change_timestamp_to_year(value):
    return value.groupby(value.index.year).sum()

def get_total_cash(ticker, past_revenue):
    stock= yf.Ticker(ticker)
    total_cash = stock.balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'].dropna() /1e9
    total_cash = change_timestamp_to_year(total_cash)
    cash_margin = (total_cash / past_revenue).mean()

    return cash_margin, total_cash

In [263]:
ticker = "AAPL"
print(f"WACC for {ticker}: {get_wacc(ticker)}")
past_revenue, rate = get_revenue_growth(ticker)
rate

WACC for AAPL: 0.0866


0.02338440383792924

In [203]:
wacc = get_wacc(ticker)

In [205]:
projection = get_revenue_projection(ticker=ticker, growth_rate= rate)

In [393]:
past_ebit, past_ebit_margin, ebit_margin= get_ebit_margin(ticker)

In [209]:
ebit = get_ebit_projection(projection, ebit_margin)

In [211]:
NOPAT = get_NOPAT(ebit)

In [409]:
past_da, da_estimite = get_depreciation_and_amortization(ticker, projection)

In [215]:
capex = get_capex(ticker, projection)

In [217]:
nwc= get_workingcapital(ticker, projection)

In [219]:
fcf = get_fcf(NOPAT, da_estimite, capex, nwc)

In [221]:
pv_fcf = get_fcf_pv(fcf, wacc)

In [223]:
tv = get_terminal_value(fcf, wacc)

In [225]:
pv_tv = get_pv_tv(tv, wacc)

In [227]:
ev = get_enterprise_value(pv_fcf, pv_tv)

In [229]:
cash, debt, net_debt = get_net_debt(ticker)

In [231]:
equity_value = get_equity_value(ev, net_debt)

In [233]:
share_count, share_price = get_implied_share_price(ticker, equity_value)

In [235]:
df= pd.concat([projection, ebit, NOPAT, da_estimite, capex,nwc, fcf, pv_fcf ],
              keys=['Revenue', 'EBIT','NOPAT (EBIT-TAX)','D&A(+)','CapEX(-)', 'Change in NWC(-)','Unlevered FCF',
                    'Present Value of FCF'], axis=1).T

In [237]:
df2 = pd.DataFrame(data=[tv, pv_tv, ev, cash, debt, equity_value, share_count, share_price], 
          index=['Terminal Value', 'PV of Terminal Value', 'Enterprise Value', 
                 'Cash (+)', 'Debt (-)', 'Equity Value', 'Shares Outstanding',
                'Implied Share Price'], columns=['2025'])

In [239]:
df

,2025,2026,2027,2028,2029
Revenue,400.180000,409.540000,419.110000,428.910000,438.940000
EBIT,121.460000,124.300000,127.200000,130.180000,133.220000
NOPAT (EBIT-TAX),95.953400,98.197000,100.488000,102.842200,105.243800
D&A(+),11.838040,12.114926,12.398024,12.687925,12.984631
CapEX(-),11.025795,11.283683,11.547356,11.817367,12.093714
Change in NWC(-),4.298355,4.398891,4.501683,4.606945,4.714678
Unlevered FCF,92.467291,94.629352,96.836985,99.105813,101.420039
Present Value of FCF,85.097819,80.146852,75.480046,71.091934,66.953806


In [430]:
df2

,2025
Terminal Value,1553.279870
PV of Terminal Value,1025.418646
Enterprise Value,1404.189103
Cash (+),29.943000
Debt (-),85.750000
Equity Value,1348.382103
Shares Outstanding,15.022100
Implied Share Price,89.759891


# Operating Data

In [365]:
past_revenue = change_timestamp_to_year(past_revenue)

In [369]:
revenue_output = pd.concat([past_revenue, projection])

In [371]:
revenue_pct_change = revenue_output.pct_change()

In [363]:
past_ebit = change_timestamp_to_year(past_ebit)

In [377]:
ebit_output = pd.concat([past_ebit, ebit])

In [403]:
ebit_margin_output = ebit_output / revenue_output

In [419]:
da_output = pd.concat([past_da, da_estimite])

In [423]:
da_rate_output = da_output / revenue_output

In [425]:
pd.DataFrame(data=[revenue_output, revenue_pct_change, ebit_output, ebit_margin_output, da_output, da_rate_output],
            index=['Revenue','Revenue %', 'EBIT', 'EBIT %','Depreciation', 'Depreciation %'])

,2021,2022,2023,2024,2025,2026,2027,2028,2029
Revenue,365.817000,394.328000,383.285000,391.035000,400.180000,409.540000,419.110000,428.910000,438.940000
Revenue %,NaN,0.077938,-0.028005,0.020220,0.023387,0.023389,0.023368,0.023383,0.023385
EBIT,108.949000,119.437000,114.301000,123.216000,121.460000,124.300000,127.200000,130.180000,133.220000
EBIT %,0.297824,0.302887,0.298214,0.315102,0.303513,0.303511,0.303500,0.303514,0.303504
Depreciation,11.284000,11.104000,11.519000,11.445000,11.838040,12.114926,12.398024,12.687925,12.984631
Depreciation %,0.030846,0.028159,0.030053,0.029268,0.029582,0.029582,0.029582,0.029582,0.029582


# Balance Sheet

In [488]:
cash_margin, past_total_cash = get_total_cash(ticker, past_revenue)

In [490]:
total_cash_estimite = projection * cash_margin

In [496]:
total_cash_output = pd.concat([past_total_cash, total_cash_estimite])

In [498]:
cash_margin_output = total_cash_output / revenue_output

In [500]:
pd.DataFrame(data=[total_cash_output, cash_margin_output],
            index=['Total Cash', 'Total Cash %'])

,2021,2022,2023,2024,2025,2026,2027,2028,2029
Total Cash,62.63900,48.304000,61.555000,65.171000,62.126822,63.579935,65.065651,66.587074,68.144203
Total Cash %,0.17123,0.122497,0.160599,0.166663,0.155247,0.155247,0.155247,0.155247,0.155247


In [450]:
stock= yf.Ticker(ticker)
stock.balance_sheet.loc['Cash Cash Equivalents And Short Term Investments'].dropna()

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       'Common Stock Equity', 'Total Capitalization',
       'Total Equity Gross Minority Interest', 'Stockholders Equity',
       'Gains Losses Not Affecting Retained Earnings',
       'Other Equity Adjustments', 'Retained Earnings', 'Capital Stock',
       'Common Stock', 'Total Liabilities Net Minority Interest',
       'Total Non Current Liabilities Net Minority Interest',
       'Other Non Current Liabilities', 'Tradeand Other Payables Non Current',
       'Long Term Debt And Capital Lease Obligation',
       'Long Term Capital Lease Obligation', 'Long Term Debt',
       'Current Liabilities', 'Other Current Liabilities',
       'Current Deferred Liabilities', 'Current Deferred Revenue',
       'Current Debt And Capital Lease Obligation',
       'Current C